### Step3-1①宿題：データ取得

In [25]:
# 必要なライブラリのインポート（アルファベット順に）
from bs4 import BeautifulSoup
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests

In [26]:
# アクセス先のUTLを指定
template_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?mt=9999999&cn=9999999&co=1&ta=13&url=/chintai/ichiran/FR301FC001/&et=9999999&smk=&shkr1=03&sc=13101&sc=13102&sc=13104&ar=030&bs=040&ct=20.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&shkr4=03&cb=0.0&page={}"

In [27]:
# ページ数を取得
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?mt=9999999&cn=9999999&co=1&ta=13&url=/chintai/ichiran/FR301FC001/&et=9999999&smk=&shkr1=03&sc=13101&sc=13102&sc=13104&ar=030&bs=040&ct=20.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&shkr4=03&cb=0.0"
res = requests.get(base_url)

# ステータスコードが正常なら実行
if res.status_code == 200:

    html_data = res.text
    soup = BeautifulSoup(html_data, 'html.parser')

    page_elements = soup.select("ol.pagination-parts a")
    page_numbers = [int(element.text) for element in page_elements] # aタグ内に記載されたページ数をリスト化
    total_pages = max(page_numbers)

print("ページ数：" + str(total_pages))

ページ数：22


In [28]:
# データフレームを作成
columns = ["物件名","住所","間取り","階数","家賃","管理費","敷金","礼金","専有面積","築年数"]
df = pd.DataFrame(columns=columns)

df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数


In [38]:
# ステータスコードが正常なら実行
if res.status_code == 200:
    html_data = res.text
    soup = BeautifulSoup(html_data, 'html.parser')

    # 各ページにアクセスして情報を取得
    for page_number in range(1, total_pages + 1):
        page_url = template_url.format(page_number)
        page_response = requests.get(page_url)

        # 必要な情報を抽出
        property_items = soup.select('.cassetteitem')

        for item in property_items:
            title = item.select_one('.cassetteitem_content-title').text  # 物件名の取得
            address = item.select_one('.cassetteitem_detail-col1').text  # 住所の取得
            age = item.select_one('.cassetteitem_detail-col3 div').text  # 築年数の取得

            floor_level = item.select_one('.js-cassette_link td:nth-of-type(3)').text.strip()  # 階数の取得
            rent_fee = item.select_one('.cassetteitem_price--rent .ui-text--bold').text  # 家賃の取得
            management_fee = item.select_one('.cassetteitem_price--administration').text  # 管理費の取得
            deposit = item.select_one('.cassetteitem_price--deposit').text  # 敷金の取得
            gratuity_fee = item.select_one('.cassetteitem_price--gratuity').text  # 礼金の取得
            occupied_area = item.select_one('.cassetteitem_menseki').text  # 専有面積の取得
            floor_plan = item.select_one('.cassetteitem_madori').text  # 間取りの取得

            # 抽出した情報をデータフレームに追加
            add_data = {
                "物件名": [title],
                "住所": [address],
                "間取り": [floor_plan],
                "階数": [floor_level],
                "家賃": [rent_fee],
                "管理費": [management_fee],
                "敷金": [deposit],
                "礼金": [gratuity_fee],
                "専有面積": [occupied_area],
                "築年数": [age]
            }

            add_df = pd.DataFrame(add_data)
            df = pd.concat([df, add_df], ignore_index=True)


In [39]:
df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数
0,都営大江戸線 東中野駅 3階建 築22年,東京都新宿区北新宿３,4SK,1階,19万円,10000円,-,19万円,80m2,築22年
1,新宿六丁目貸一戸建,東京都新宿区新宿６,4K,2階,17万円,-,34万円,17万円,58.66m2,築51年
2,グランドフェニックス,東京都新宿区西早稲田３,2LDK,5階,17.5万円,16000円,17.5万円,-,68.38m2,築28年
3,フィオーレ月島,東京都中央区佃２,2LDK,5階,18.5万円,-,18.5万円,37万円,54.26m2,築31年
4,ライオンズマンション日本橋,東京都中央区日本橋蛎殻町１,3DK,9階,13万円,5000円,13万円,6.5万円,61.36m2,築42年
...,...,...,...,...,...,...,...,...,...,...
2635,ＪＲ京葉線 八丁堀駅 13階建 築20年,東京都中央区湊２,2DK,2階,19.2万円,-,19.2万円,19.2万円,50.38m2,築20年
2636,東京メトロ有楽町線 新富町駅 8階建 築27年,東京都中央区築地７,2LDK,5階,17.5万円,10000円,17.5万円,17.5万円,48.41m2,築27年
2637,勝どきビュータワー,東京都中央区勝どき１,2LDK,47階,18万円,20000円,18万円,9万円,58.8m2,築14年
2638,東京メトロ有楽町線 月島駅 5階建 築31年,東京都中央区佃２,2LDK,5階,18.5万円,-,18.5万円,37万円,54.26m2,築31年


In [40]:
# Googleスプレッドシートの認証情報を取得
SCOPES = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = "tech0-step3_1_1-suumo-jsonkey.json"

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)
gs = gspread.authorize(credentials)

SPREADSHEET_KEY = "1rRtCz79tHmcBlmRGqbQDgwxgA_P8822bFhcb-k6MxQs"

workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("20231203")

In [41]:
# データフレームをGoogleスプレッドシートに転記
set_with_dataframe(worksheet,df,include_index=True)

### Step3-1② 宿題：クレンジングと重複削除

#### クレンジング

In [81]:
# 必要なライブラリのインポート
import re

In [82]:
# 関数定義：数値化
def extract_numeric(value):
    numeric_values = re.findall(r'\d+\.?\d*', value)
    if numeric_values:
        return float(numeric_values[0])  # 文字列から浮動小数点数に変換
    else:
        return None

In [83]:
# ステータスコードが正常なら実行
if res.status_code == 200:

    # 各ページにアクセスして情報を取得
    for page_number in range(1, total_pages + 1):
        page_url = template_url.format(page_number)
        page_response = requests.get(page_url)

        if page_response.status_code == 200:
            soup = BeautifulSoup(page_response.text, 'html.parser')

            # 必要な情報を抽出
            property_items = soup.select('.cassetteitem')

            for item in property_items:
                title = item.select_one('.cassetteitem_content-title').text  # 物件名の取得
                address = item.select_one('.cassetteitem_detail-col1').text  # 住所の取得

                age = item.select_one('.cassetteitem_detail-col3 div').text  # 築年数の取得
                re_age = extract_numeric(age) # 築年数の数値化
                re_age = int(re_age) if re_age is not None else None

                floor_level = item.select_one('.js-cassette_link td:nth-of-type(3)').text.strip()  # 階数の取得
                re_floor_level = extract_numeric(floor_level) # 階数の数値化
                re_floor_level = int(re_floor_level) if re_floor_level is not None else None

                rent_fee = item.select_one('.cassetteitem_price--rent .ui-text--bold').text  # 家賃の取得
                re_rent_fee = extract_numeric(rent_fee) # 家賃の数値化

                management_fee = item.select_one('.cassetteitem_price--administration').text  # 管理費の取得
                re_management_fee = extract_numeric(management_fee) # 管理費の数値化

                deposit = item.select_one('.cassetteitem_price--deposit').text  # 敷金の取得
                re_deposit = extract_numeric(deposit) # 敷金の数値化

                gratuity_fee = item.select_one('.cassetteitem_price--gratuity').text  # 礼金の取得
                re_gratuity_fee = extract_numeric(gratuity_fee) # 礼金の数値化

                occupied_area = item.select_one('.cassetteitem_menseki').text  # 専有面積の取得
                re_occupied_area = extract_numeric(occupied_area) # 専有面積の数値化

                floor_plan = item.select_one('.cassetteitem_madori').text  # 間取りの取得

                # 抽出した情報をデータフレームに追加
                add_re_data = {
                    "物件名": [title],
                    "住所": [address],
                    "間取り": [floor_plan],
                    "階数": [re_floor_level],
                    "家賃": [re_rent_fee],
                    "管理費": [re_management_fee],
                    "敷金": [re_deposit],
                    "礼金": [re_gratuity_fee],
                    "専有面積": [re_occupied_area],
                    "築年数": [re_age]
                }

                add_re_df = pd.DataFrame(add_re_data)
                re_df = pd.concat([re_df, add_re_df], ignore_index=True)

In [85]:
re_df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数
0,都営大江戸線 東中野駅 3階建 築22年,東京都新宿区北新宿３,4SK,1階,19万円,10000円,-,19万円,80m2,築22年
1,新宿六丁目貸一戸建,東京都新宿区新宿６,4K,2階,17万円,-,34万円,17万円,58.66m2,築51年
2,グランドフェニックス,東京都新宿区西早稲田３,2LDK,5階,17.5万円,16000円,17.5万円,-,68.38m2,築28年
3,フィオーレ月島,東京都中央区佃２,2LDK,5階,18.5万円,-,18.5万円,37万円,54.26m2,築31年
4,ライオンズマンション日本橋,東京都中央区日本橋蛎殻町１,3DK,9階,13万円,5000円,13万円,6.5万円,61.36m2,築42年
...,...,...,...,...,...,...,...,...,...,...
680,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35
681,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35
682,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35
683,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35


#### 重複削除

In [86]:
# 「住所」「間取り」「階数」「家賃」「専有面積」が一致する行を削除
re_df.drop_duplicates(subset=['住所', '間取り', '階数', '家賃', '専有面積'], inplace=True)

In [87]:
re_df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数
0,都営大江戸線 東中野駅 3階建 築22年,東京都新宿区北新宿３,4SK,1階,19万円,10000円,-,19万円,80m2,築22年
1,新宿六丁目貸一戸建,東京都新宿区新宿６,4K,2階,17万円,-,34万円,17万円,58.66m2,築51年
2,グランドフェニックス,東京都新宿区西早稲田３,2LDK,5階,17.5万円,16000円,17.5万円,-,68.38m2,築28年
3,フィオーレ月島,東京都中央区佃２,2LDK,5階,18.5万円,-,18.5万円,37万円,54.26m2,築31年
4,ライオンズマンション日本橋,東京都中央区日本橋蛎殻町１,3DK,9階,13万円,5000円,13万円,6.5万円,61.36m2,築42年
...,...,...,...,...,...,...,...,...,...,...
680,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35
681,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35
682,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35
683,ライプワン目白 手割引 振分 シェア可 子供可 1階カクヤス,東京都新宿区下落合４,2LDK,5,12.8,10000.0,12.8,12.8,46.2,35


### Step3-1③：スプレッドシートに格納

In [88]:
# Googleスプレッドシートの認証情報を取得
SCOPES = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = "tech0-step3_1_1-suumo-jsonkey.json"

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)
gs = gspread.authorize(credentials)

SPREADSHEET_KEY = "1rRtCz79tHmcBlmRGqbQDgwxgA_P8822bFhcb-k6MxQs"

workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("20240111")

In [89]:
# データフレームをGoogleスプレッドシートに転記
set_with_dataframe(worksheet,re_df,include_index=True)